In [1]:
import gc

import numpy as np
import pandas as pd
from random import sample

import hyperopt
from hyperopt import hp, Trials
from hyperopt.pyll.base import scope

from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from coremdlr.models import NetworkModel
from coremdlr.config import defaults

from coremdlr.datasets import WellLoader, FaciesDataset
from coremdlr.datasets.utils import infer_test_wells

import matplotlib.pyplot as plt
%matplotlib inline

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3

In [2]:
DEEPTEN_ARGS = {
    'feature' : 'image',
    'network' : 'deepten',
    'summary' : True,
    'sequence_size' : 12,
    'network_args' : {
        'backbone_cnn' : 'vgg16',
        'conv1x1' : 40,
        'encode_K' : 24,
        'apply_rowwise' : True,
        'lstm_features' : None,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
        'clipnorm' : 1.0
    }
}

IMAGE_ARGS = {
    'image_width' : 200,
    'crop_method' : 'center',
    'downsample' : 2
}


fit_args = {
    'batch_size' : 4,
    'epochs' : 25,
    'class_weighted' : True
}

CHKPT_FNAME = 'temp_best_weights.h5'

In [3]:
labels_dict = {str(i) : str(i) for i in range(8)}

labeled_wells = [
    '204-19-3A',
    '204-19-6',
    '204-20-3',
    '204-20-6a',
    '204-24a-6'
]
    
pred_dfs = []

def pred_results_df(model_config):
    
    for i, test_well in enumerate(labeled_wells):
        
        gc.collect()
        
        dset = FaciesDataset([w for w in labeled_wells if w != test_well], [test_well], 
                             labels_ext='_labelsII.npy',
                             lithology_classes=labels_dict,
                             features=['image'],
                             image_args=IMAGE_ARGS)
        
        dset.load_or_generate_data()
        
        model = NetworkModel(dset, model_args=model_config)
        
        chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_acc', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
        earlystop_callback = EarlyStopping(monitor='val_acc', patience=5)
        fit_args['callbacks'] = [chkpt_callback, earlystop_callback]
        
        model.fit(dset, verbose=False, **fit_args)
        
        model.network.load_weights(CHKPT_FNAME)
        
        #print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name)
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
        del dset; del model;
        
    return pd.concat(pred_dfs)

In [4]:
preds_df_3 = pred_results_df(DEEPTEN_ARGS)
preds_df_3.to_csv('image_deepten_results_labelsII.csv')

[b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7']
Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (1879,)), ('top', (1879,)), ('base', (1879,)), ('image', (1879, 32, 200, 3))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (11392,)), ('top', (11392,)), ('base', (11392,)), ('image', (11392, 32, 200, 3))]
Loading Well:  204-20-6a  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (3536,)), ('top', (3536,)), ('base', (3536,)), ('image', (3536, 32, 200, 3))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (12152,)), ('top', (12152,)), ('base', (12152,)), ('image', (12152, 32, 200, 3))]


/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Loading Well:  204-19-3A  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (8946,)), ('top', (8946,)), ('base', (8946,)), ('image', (8946, 32, 200, 3))]


/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384, 200, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 384, 200, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 384, 200, 64) 36928       block1_conv1[0][0]               
_______________________________________________________

...

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (12152,)), ('top', (12152,)), ('base', (12152,)), ('image', (12152, 32, 200, 3))]


/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 384, 200, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 384, 200, 64) 1792        input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 384, 200, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 192, 100, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

1071/1072 [============================>.] - ETA: 0s - loss: 1.1839 - acc: 0.4770
Epoch 00002: val_acc improved from 0.05077 to 0.29735, saving model to temp_best_weights.h5
1072/1072 [==============================] - 100s 94ms/step - loss: 1.1835 - acc: 0.4771 - val_loss: 1.5983 - val_acc: 0.2973
Epoch 3/25
1071/1072 [============================>.] - ETA: 0s - loss: 1.1681 - acc: 0.4765
Epoch 00003: val_acc did not improve from 0.29735
1072/1072 [==============================] - 100s 93ms/step - loss: 1.1677 - acc: 0.4767 - val_loss: 1.3959 - val_acc: 0.1299
Epoch 4/25
1071/1072 [============================>.] - ETA: 0s - loss: 1.1626 - acc: 0.4833
Epoch 00004: val_acc improved from 0.29735 to 0.34643, saving model to temp_best_weights.h5
1072/1072 [==============================] - 100s 94ms/step - loss: 1.1624 - acc: 0.4833 - val_loss: 8.3211e-06 - val_acc: 0.3464
Epoch 5/25
1071/1072 [============================>.] - ETA: 0s - loss: 1.1675 - acc: 0.4877
Epoch 00005: val_acc di

In [5]:
accuracy_score(preds_df_3.y_pred.values, preds_df_3.y_true.values)

0.3513150831991412

In [6]:
from sklearn.metrics import confusion_matrix

confusion_matrix(preds_df_3.y_true.values, preds_df_3.y_pred.values)

array([[10320,   304,     6,    31,    11,    92,   161],
       [ 6223,   426,    37,    52,    20,   118,   558],
       [  397,     4,     0,     0,     0,     0,    14],
       [ 1251,    37,     6,     5,     0,    12,   102],
       [  767,    11,     2,     1,     1,    15,    37],
       [ 3432,   101,     3,     3,     1,    23,  1281],
       [ 6570,    27,     3,    13,     2,    48,  1006]])